# Microgrids simulator showcase

Demo of the operational & economic simulation of a Microgrid with Solar power, a battery and a dispatchable generator (using [Microgrids.py](https://github.com/Microgrids-X/Microgrids.py)).

**Intructions**: Use the Run button (▶️) or `Shift+Enter` to run selected cells.

Remark: although this app is displayed in a web browser, all data is stored and processed *locally on your computer* (unlike typical cloud apps) thanks to [JupyterLite](https://jupyterlite.readthedocs.io/en/latest/). Consequences: 1) your privacy is respected, and 2) if you want a record of your experiment, use "File/Download". 

In [1]:
%run -i microgrid_showcase_setup.py

microgrid_showcase_setup.py
Showcase setup complete.


## Interactive energy mix

Experiment starting from zero PV and zero battery:

1. Start by first increasing PV power
    - at first this reduce the usage of the generator
    - but beyond ~2000 kW, there is more and more spilled energy
2. Then, for a PV power of 5000 kW, increase battery energy to reduce spilled energy
    - at first, this reduces spilled energy and therefore reduces further generator usage
    - but beyond ~8000 kWh, it requires a higher and higher capacity to get an effect

With appropriate settings, you should find a Levelized Cost of Electricity below 0.30 \\$/kWh (while it is 0.35 \\$/kWh without PV and battery).

In [2]:
interactive(interactive_energy_mix, PV_power=(0.0, 10e3, 500), Batt_energy=(0.0, 15e3, 1e3))

interactive(children=(FloatSlider(value=0.0, description='PV_power', max=10000.0, step=500.0), FloatSlider(val…

### Bonus: undersized generator – degraded quality of service

You can experiment with an *undersized* dispatchable generator (undersized: less than max load).
This yields some *“load shedding”*, that is a *degraded quality of service* where the desired electricity consumption is not satisfied at all times.

Then you can witness how the solar plant together with the battery can *improve* this quality of service (only partially though).

**Intructions**: 
1. Run the cell to change the value of `power_rated_gen`. This is a global variable which sets the size of the generator.
2. Go back to the interactive plot above and move the slider to update the computation with the new generator size

In [3]:
power_rated_gen = 1000.  # < 1000 to see some significant load shedding (kW)